# Quantum One-Time Pad: Assignment

*<font color='red'> Read everything carefully before starting to solve questions provided at the end of the notebook </font>*

**Some Hints:**
- Its always useful to display the quantum circuit and check whether the gates are applied as desired. Also mentally (or on a pen and paper) check the gate operations you apply.
- Square of Hadamard, Z and X gates is equal to Identity.

#### <font color='yellow'> Cryptography is a technique used for secure communication between two parties, where unauthorized users and malicious attackers can be present.</font>

In cryptography, there are majorly two processes:
- **Encryption** 
    - The process of converting essential information / message into unreadble encrypted form (also called *cipher text*) using keys (additional data string).
    - It is performed at the sender's end.
 

- **Decryption** 
    - The process of decoding the message by converting encrypted message to original message using the same key (*symmetric key cryptography*) or different key (*assymmetric key cryptography*).
    - It is performed at the receiver's end.

 You can read [here](https://www.geeksforgeeks.org/classical-cryptography-and-quantum-cryptography/?ref=rp) and watch [this](https://www.khanacademy.org/computing/computer-science/cryptography/crypt/v/intro-to-cryptography) for a little more explanation on Cryptography.



## Classical One-Time Pad

One-Time Pad is an example for Symmetric key Cryptography, as the same key is used for both encryption and decryption. The well-known RSA encryption you might have heard of is an example for Assymmetric key Cryptography.

In **One-Time Pad** each bit of the message is encrypted using the corresponding bit from a randomly generated key (one-time pad) using modular addition. 

Following are the necessary requirements to make the encrypted text **impossible** to decrypt:

1. The key must be at least as long as the message.
2. The key must be random and independent of the message, entirely sampled from a non-algorithmic source such as a hardware random number generator. 
3. The key must never be reused in whole or in part.
4. The key must be kept completely secret by the communicating parties.

<font color= 'red'> **Let's understand one-time pad with the help of an example:** </font>

* Anshu wants to share a message with Bharat. She converts the message to binary form (say using [ASCII Table](https://www.ascii-code.com/))
* To encrypt the mesage, she tosses a fair coin, and based on the result she generates a key of 0's (Head) and 1's (Tail). The key is as long as the message and thus she has to toss the coin `len(message)` times to generate the full key.
* She does a bitwise `xor` operation between the message bits and key bits. The result is an encrypted message.
* Bharat uses the **same key** to apply `xor` on the encrypted message received and if this has not been tampered he'll get the original message back. 

The idea here is straightforward: message $\oplus$ key $\oplus$ key = message

### Example:
Anshu wants to send Bharat a message that contains information about their next meetup, say 10PM. She encrypts this message using one-time pad. 

She converts the time to its binary representation using ASCII character encoding. "10PM" is '00110001001100000101000001001101' in binary. Anshu needs a random key of the same string length (32 bits) to encrypt her message. She uses a random number generator to generate this key and does a bitwise `xor` operation between the binary message and the key to get the encrypted message which is then decoded by Bharat using the same key. 

In [1]:
#Generating random key of the same length as the message

from random import randrange
lst=[]
i=0

while i<31:
    lst.append(randrange(2))
    i=i+1
    
key=int(''.join(map(str, lst)), 2) #converting key to binary format and then to the corresponding integer

print(key)

from random import randrange

m= '00110001001100000101000001001101'   #message in binary

message=int(m ,2) #convert message to integer to apply xor operation

#Encryption

encrypted_message=message^key #encrypt the message using generated random key by applying xor operator

print('Message =', bin(message)[2:].zfill(32))
print('Key =', bin(key)[2:].zfill(32))
print('Ciphertext =', bin(encrypted_message)[2:].zfill(32))


#Bharat then decrypts the encrypted text by applying the same key 

decrypted_message=encrypted_message^key
print('Decrypted Message =', bin(encrypted_message)[2:].zfill(32))

  
if message == decrypted_message:  
    print("Protocol is correct!")  
else:  
    print("Protocol is incorrect!")


#if needed:
##bin() converts the integer to binary
#[2:] removes the initial "0b" that is added at the start of the binary string (string slicing)
#zfill(n) adds back leading zeros so that length of string is equal to n

809417820
Message = 00110001001100000101000001001101
Key = 00110000001111101011110001011100
Ciphertext = 00000001000011101110110000010001
Decrypted Message = 00000001000011101110110000010001
Protocol is correct!


Thus Bharat has successfully decoded his message.

## Quantum One-Time Pad

Quantum One-Time Pad is the quantum version of classical One-Time Pad. 

Suppose Anshu wants to send a message to Bharat using qubits, such that Chaman cannot decrypt it.

By using a randomly generated classical key, say $ k \in\{0,1\} $, for encryption Anshu will apply $X$ gate to each of her qubit if $k=1$ and do nothing if $k=0$:
$$\ket{E} = X_k\ket{M}$$
where
$$X\ket{M} = \ket{M \oplus 1}$$

To decrypt Bharat will use the same key and re-apply $X$ gate if $k=1$ and do nothing if $k=0$

$$\ket{M} = X_k\ket{E}$$

Note: $ \ket{M}=$ Orginal message, $\ket{E}=$ Encrypted message

In [2]:
# Quantum One Time Pad in the standard basis $\ket{0}, \ket{1}$

# Encryption

from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer
from random import randrange

# Enter your message as binaries in a list
message = [0,0,1,1,0,0,0,1]  #say this is a 8-bit message Anshu wants to send to Bharat

# create a quantum curcuit for Anshu's qubits.
qregA = QuantumRegister(len(message)) 
cregA = ClassicalRegister(len(message)) 
mycircuitA = QuantumCircuit(qregA,cregA, name= 'Anshu')

# apply x-gate to change initial states from 0 to 1, preparing the given message 
for m in range(len(message)):
    if message[m]==1:
        mycircuitA.x(qregA[m])

# barrier for a better visualization
mycircuitA.barrier()

# create random key and apply x gates and encrypt message
key=[] 
for i in range(len(message)):
    a=randrange(2)
    key.append(a)
    if a==1:
        mycircuitA.x(qregA[i])


print('Preshared Key between Anshu and Bharat:', key)
        
mycircuitA.barrier()
mycircuitA.measure(qregA,cregA)
mycircuitA.draw()

# execute the circuit
job = execute(mycircuitA,Aer.get_backend('qasm_simulator'))
encryption = job.result().get_counts(mycircuitA)

# this converts the measurement result string (encryption) into a list
encrypted_message=list(map(int,[*list(encryption.keys())[0]]))
print(encrypted_message)

# we reverse the list since the quantum simulator considers our MSB as LSB
encrypted_message.reverse()
print()
print("Message:", message)
print("Key:", key)
print("Encrypted message to be sent to Bharat: ", encrypted_message)
print() 

# Decryption


# create a quantum curcuit for Bharat's qubits.

qregB = QuantumRegister(len(message)) 
cregB = ClassicalRegister(len(message)) 
mycircuitB = QuantumCircuit(qregB,cregB, name= 'Bharat')


# apply x-gate to change initial states from 0 to 1, preparing the state with encrypted message
for m in range(len(encrypted_message)):
    if encrypted_message[m]==1:
        mycircuitB.x(qregB[m])
        

# use the same random key and apply x gates to decrypt message
for i in range(len(key)):
    if key[i]==1:
        mycircuitB.x(qregB[i])
          

# Perform Measurement
mycircuitB.barrier()
mycircuitB.measure(qregB,cregB)

mycircuitB.draw()


# execute the circuit
job = execute(mycircuitB,Aer.get_backend('qasm_simulator'))
decryption = job.result().get_counts(mycircuitB)

# this converts the measurement result string (decryption) into a list
decrypted_message=list(map(int,[*list(decryption.keys())[0]]))

# we reverse the list since the quantum simulator considers our MSB as LSB
decrypted_message.reverse()
print("Message decrypted by Bharat:", decrypted_message)
print()  
  
if message == decrypted_message:  
    print("Protocol is correct! Sent Message==Received Message")  
else:  
    print("Protocol is incorrect!")

Preshared Key between Anshu and Bharat: [1, 0, 0, 1, 1, 0, 1, 0]
[1, 1, 0, 1, 0, 1, 0, 1]

Message: [0, 0, 1, 1, 0, 0, 0, 1]
Key: [1, 0, 0, 1, 1, 0, 1, 0]
Encrypted message to be sent to Bharat:  [1, 0, 1, 0, 1, 0, 1, 1]

Message decrypted by Bharat: [0, 0, 1, 1, 0, 0, 0, 1]

Protocol is correct! Sent Message==Received Message


So you see that this works. But we haven't really used the power of 'quantum' as such here. The above code does exactly what the classical one-time pad does, but using quantum circuits. 

**Note: We worked with $Z-basis$ for the encryption/decryption above.** 
 Some points that might help you realize this further:
- We use $\ket{0}$ and $\ket{1}$ as computational basis states. These vectors are eigenvectors of the $Z$ gate (or Pauli Z matrix). (Check!!)
    - In this basis $X$ gate is the $NOT$ gate. (it flips $\ket{0}$ to $\ket{1}$ and vice-versa)
- You can have other basis states, like $\ket{+}$ and $\ket{-}$. You can get to this basis by applying Hadamard on computational basis states.
    - In this basis ___ is the $NOT$ gate. (it flips $\ket{+}$ to $\ket{-}$ and vice-versa) (<font color='red'>fill in the blank </font>)


<font style="color:darkorange;font-weight:bold;"> <h2>Main Task 1</h2> </font>
- #### Using Hadamard Basis and replacing $\ket{0}$ with $\ket{+}$ and $\ket{1}$ with $\ket{-}$, encrypt the same message with $X$ gates and verify that the protocol is incorrect.
- #### Why do you think that using X gates for encryption in Hadamard Basis won't work?

<font style="color:darkorange;font-weight:bold;"> <h2>Main Task 2</h2> </font>

#### Develop a complete Quantum One Time Pad and ensure that your protocol is correct. Here is the protocol to implement this: 

- <b>$X$</b> gate is used to encrypt in Standard basis.

- <b>$Z$</b> gate is used to encrypt in Hadamard basis.

- By using a classical key (generated randomly), Anshu will apply both or either of gates (X and Z) randomly to her qubit. Before applying $Z$ gate as the NOT operator you need to make sure that the qubit to which it is applied is in the Hadamard basis.

- For encryption,

$$\ket{E}= X_{k1}Z_{k2}\ket{M}$$
where, 
$$X\ket{M} =  \ket{M \oplus 1}$$

$$Z\ket{M} =  (-1)^{M}\ket{M}$$

- For decryption,

$$\ket{M} = Z_{k2}X_{k1}\ket{E}=Z_{k2}X_{k1}(X_{k1}Z_{k2}\ket{M})=\ket{M}$$

where $X_{k1}X_{k1}=I$ and $Z_{k2}Z_{k2}=I$.  (Square of Pauli matrices is Identity)

- You can see here that for the complete quantum one-time pad, Anshu will need to use <b>two bits</b> of the key <b>k<sub>1</sub>k<sub>2</sub></b> to encode one quantum state $ \ket{Ψ} $: one bit for X-gate and the second for Z-gate. So, the **key must be twice as long as the message** unlike the example shown above.


### The below notes will help in $Main Task 2$

As you can see from the result of $Main Task 1$, when Anshu switched to Hadamard basis, encoding with $X$ gate didn't work any more. The solution was to use $Z$ gate, which is a bit-flip operation for Hadamard basis. Ok, so we figured out what flips bits in Hadamard basis.

- In the example code above, notice that we had performed a measurement on Anshu's circuit to get the encrypted message which was used by Bharat for decryption (given that he knows the key). But if Hadamard basis is used (and also $Z$ gate for flipping according to the generated key), measurement results result in multiple encryptions (as you saw in Main Task 1) because of the superpositions. 

(Reason:  Measurement results in qiskit is always with respect to the computational basis states ($\ket{0}$ and $\ket{1}$). That's why you got multiple encryptions in $Task 1$ and thus decryption was not possible)

So to tackle it we won't do a measurement on Anshu's side and send the final quantum state (obtained after Anshu's operations to qubits, some of which are in Hadamard and some in computational basis) to Bharat directly using the function defined below.

In [8]:
# DO NOT MODIFY THE FUNCTION BELOW (unless you understand every line of it). You are not expected to understand this code, just use it.

from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer
from random import randrange

#Source for SendState: awards/teach_me_qiskit_2018/cryptography/Cryptography.ipynb (the code taken from here has been modified a bit for this assignment, to incorporate Z gate)

def SendState(qc1, qc2, qc1_name):
    ''' This function takes the output of a circuit qc1 (made up only of x,
        h and z gates and initializes another circuit qc2 with the same state
    ''' 
    
    # Quantum state is retrieved from qasm code of qc1
    qs = qc1.qasm().split(sep=';')[4:-1]

    # Process the code to get the instructions
    for index, instruction in enumerate(qs):
        qs[index] = instruction.lstrip()

    for instruction in qs:
        if instruction[0] == 'x':
            if instruction[5] == '[':
                old_qr = int(instruction[6:-1])
            else:
                old_qr = int(instruction[5:-1])
            qc2.x(qreg[old_qr])
        elif instruction[0] == 'h':
            if instruction[5] == '[':
                old_qr = int(instruction[6:-1])
            else:
                old_qr = int(instruction[5:-1])
            qc2.h(qreg[old_qr])
        elif instruction[0] == 'z':
            if instruction[5] == '[':
                old_qr = int(instruction[6:-1])
            else:
                old_qr = int(instruction[5:-1])
            qc2.z(qreg[old_qr])
        elif instruction[0] == 'm': # exclude measuring:
            pass
        # else:
        #     raise Exception('Unable to parse instruction')  # I commented this from the original code (there's a specific reason, do you see what")

Once Bharat receives Anshu's encryption, given that he has the right key he would be able to decrypt the message as provided in the protocol in $MainTask2$

#### An example to show how `SendState()` function works

In [15]:
# Defining the first circuit

qreg= QuantumRegister(2)
creg= ClassicalRegister(2)
circuit1 = QuantumCircuit(qreg, creg, name='1')

#Doing operations on circuit1

circuit1.h(qreg[0])
circuit1.x(qreg[1])

# Defining the first circuit
circuit2 = QuantumCircuit(qreg, creg, name='2')

SendState(circuit1, circuit2, '1') #the final state of circuit '1' is sent to the circuit '2'

circuit2.draw()   #notice how the operations got transferred from circuit 1 to circuit 2 after executing this line

┌───┐
q21_0: ┤ H ├
       ├───┤
q21_1: ┤ X ├
       └───┘
c14: 2/═════

<h3> Conclusion </h3>

We can see that One-time Pad protocol can be implemented in quantum communication. Quantum one-time pad protocol is correct and secure, even if we assume that our eavesdropper has powerful quantum computer with infinite quantum memory and can listen the channel.

**Drawbacks:**
<ul>
    <li> The key we are using is classical, which means that it is not truly random.
<li> The key is an (overly) important resource. For quantum One-Time Pad, we need a key that is twice as long as the message, and Anshu and Bharat will be unable to communicate once they run out of keys.
</ul>

<b>These two problems are addressed by Quantum Key Distribution Techniques, which we'll be discussing later.</b>

## References


- A major part of the code for `SendState()` function has been taken from [here](https://github.com/CQCL/qiskit-tutorial/tree/master)
- QWorld resources on [Quantum Cryptography](https://gitlab.com/qworld/qeducation/qbook101)
- Some notes and intext reading suggestions from [here](https://www.geeksforgeeks.org/classical-cryptography-and-quantum-cryptography/?ref=rp )